In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys,os
sys.path.append('..')
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
from backtester.swarms.swarm import Swarm
from backtester.exoinfo import EXOInfo

import pandas as pd
import numpy as np
import scipy

In [ ]:
TMQRPATH = os.getenv("TMQRPATH", '')

swarms = {}
for file in os.listdir(os.path.join(TMQRPATH, 'swarms')):
    swarm_name = file.replace('.swm','')
    print('Loading: '+ swarm_name)
    swarms[swarm_name] = Swarm.load(os.path.join(TMQRPATH, 'swarms', file))

### Available swarms

In [ ]:
print("Swarms list:")
for swarm_name in sorted(swarms.keys()):
    print(swarm_name)

### Available EXOs

In [ ]:
passive_exo = {}


print('Passive EXO list')
for matfile in os.listdir(os.path.join(TMQRPATH, 'mat')):    
    if 'strategy_' in matfile and '.mat' in matfile:
        exo = EXOInfo.from_matfile(os.path.join(TMQRPATH, 'mat', matfile))
        passive_exo[exo.exo_name()] = exo.exo_price_index()    

for exo in sorted(passive_exo.keys()):
    print(exo)

# Campaign composition

Copy/paste EXO or swarm names to the 'campaign_exposure' dictionary from lists above

In [ ]:
campaign_exposure = {
     'CL_PutSpread_Short_IchimokuCloud': 1,     ## .5,90
     'CL_PutSpread_Short_SwingPoint': 1,        ## .5,90
     'CL_PutSpread_Short_BollingerBands': 1,    ## .5,90
     
     'CL_PutSpread_Long_SwingPoint': 1,         ## .5,90 
     'CL_PutSpread_Long_RenkoPatterns_no_exit_on_patterns': 1,         ## .5,90 
     'CL_PutSpread_Long_PointAndFigurePatterns': 1,         ## .5,90 
    
    
     #
     # Passive EXO positions
     #
     #'EP_BullishCollar_EXO': -1, # Short 1 EXO units
     
}

In [ ]:
#def campaign_equity(swarms, exposure):

campaign_dict = {}
campaign_stats = {'NetProfit': 0.0, 'TradesCount': 0, 'CommissionSum': 0.0}

for camp_name, exposure in campaign_exposure.items():
    if camp_name in swarms:
        swarm_name = camp_name
        smgr = swarms[swarm_name]
        campaign_dict[swarm_name] = smgr.picked_equity
        
        # TODO: implement swarm statistics
        '''
        s = smgr.picked_stats        
        campaign_stats['NetProfit'] += s['NetProfit']
        campaign_stats['TradesCount'] += s['TradesCount']
        campaign_stats['CommissionSum'] += s['CommissionSum']
        '''
        
    elif camp_name in passive_exo:
        exo_equity = passive_exo[camp_name]
        campaign_dict[camp_name] = exo_equity.diff() * exposure
    
    
campaign_portfolio = pd.DataFrame(campaign_dict)
campaign_equity = campaign_portfolio.sum(axis=1)

### Campaign members equities

In [ ]:
figsize(10,10)
campaign_portfolio.plot()

### Cumulative campaign equity

In [ ]:
campaign_portfolio.sum(axis=1).plot()

# Campaign statistics

In [ ]:
print(campaign_stats)